# Data Extraction, Transformation and Loading

__This Notebook does the following:__<br>
1. Loops through each tail number directory and each set of flights contained within.
2. Reads each flight from the matlabl \*.mat files (Extract)
3. Creates metadata for each flight from matlab data
4. Add metadata to each flight for starttime and endtime 
5. Creates new directory with \_parquet suffix to save everything in.
5. Saves metada as JSON file (original_file_name.json)
6. Create single GMT time reference value in secs
7. Add NaN's to fill in all data that is less than 16Hz frequency. (later NaNs are filled in with linear interpolation or backfilled. done in )
8. Create a pandas dataframe for each flight (Transform)
9. Save DF as parquet file (Load)

The size of the NASA dataset was too large for the scope of this paper so only the "Tail_687_1" dataset
was used.  Future work will use additional datasets.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import pandas as pd
#import dateutil
import os
import glob
from collections import defaultdict
from datetime import timedelta, date
import json

%matplotlib inline

## Downloading the data
    I ran these commands below in a Win10 Powershell

In [2]:
#!wget "https://c3.nasa.gov/dashlink/static/media/dataset/Tail_687_1.zip"  -outfile "Tail_687_1.zip"
##unzipping the file
#!Expand-Archive -Path  "Tail_687_1.zip"

In [4]:
cwd = os.getcwd()
print(cwd)


dirlist = glob.glob('Tail*[0-9]')
dirlist

/ssd/MBSE/phd/research/work_folder/NASA


['Tail_654_4',
 'Tail_660_7',
 'Tail_666_3',
 'Tail_675_8',
 'Tail_683_1',
 'Tail_687_1']

# Creating and Saving Metadata for all the files in the directory

This is not necessary to do unless you would like to see it.
The sensor fusion scripts later do not use this information

In [55]:
cwd = os.getcwd()
print(cwd)


dirlist = glob.glob('Tail*[0-9]')
for fdir in dirlist:
  
    #change directories to the one with all the .mat files
    os.chdir(f'{cwd}/{fdir}')
    
    #write the json file into a new dir for parquet files
    pdir = os.path.join(cwd,f'{fdir}_parquet')
    
    #create the parquet directory unless it's already created
    try:
        os.mkdir(pdir)
    except OSError as error:
        print(error)
        print('directory already exists...moving on.')
    
    metadata = []
    metadata = defaultdict(dict)
    
    filelist = glob.glob(f'*.mat')
    print(f'Collecting metadata from flight files in {fdir}')
    
    for f in filelist:
        d = loadmat(f)
        year   = d['DATE_YEAR'][0][0][0][:]
        month  = d['DATE_MONTH'][0][0][0][:]
        day    = d['DATE_DAY'][0][0][0][:]
        hour   = d['GMT_HOUR'][0][0][0][:]
        minute = d['GMT_MINUTE'][0][0][0][:]
        second = d['GMT_SEC'][0][0][0][:]
        ys = min(np.unique(year))
        mth = min(np.unique(month))
        ds = min(np.unique(day))
       
        #need to add logic to ensure start and stoptimes are valid
        hs = hour[0][0].astype(float)
        ms = minute[0][0].astype(float)
        ss = second[0][0].astype(float)
        he = hour[-1][0].astype(float)
        me = minute[-1][0].astype(float)
        se = second[-1][0].astype(float)
        starttime = timedelta(hours=hs, minutes=ms, seconds=ss)
        stoptime  = timedelta(hours=he, minutes=me, seconds=se)
        
       
        #check to see if invalid year,month,day are the only option.
        #if so, this implies that the aircraft is on the ground the entire recording.
        if ys == 2165: 
            airbornedata = False
        else:
            airbornedata = True
        
        #calculated metadata
        metadata[f]['AirborneData'] = airbornedata
        metadata[f]['StartDate'] = {}
        metadata[f]['StartDate']['Year']  = ys.astype(str)
        metadata[f]['StartDate']['Month'] = mth.astype(str)
        metadata[f]['StartDate']['Day']   = ds.astype(str)
        #metadata[f]['TotalTime'] = stoptime - starttime  #need to ensure I'm not grabbing bad time
        
    
        # Stored Metadata in each file
        for k in d.keys():
            #don't print the first "__xxxx__" parameters
            if type(d[k]) == np.ndarray:
                u = d[k]["Units"][0,0] 
                
                #check to see if the "units" are blank and set to '<none>'
                if u.shape[0] == 0: u = ["<none>"]
                
                metadata[f][k] = {}
                metadata[f][k]['Units']       = u[0]
                metadata[f][k]['Rate']        = d[k]["Rate"][0,0][0,0].astype('float')
                metadata[f][k]['Alpha']       = d[k]["Alpha"][0,0][0]
                metadata[f][k]['Description'] = d[k]["Description"][0,0][0]
                #print(f'{k:12s}Units: {str(metadata[file][k]["Units"]):10s}Rate: {str(metadata[file][k]["Rate"]):5s}',
                        #f'Alpha: {metadata[file][k]["Alpha"]:11s} Description: {metadata[file][k]["Description"]:10s}')
                
    
    #writing to a json file named after the flight directory in the parquet directory
    pjfile = os.path.join(cwd,f'{fdir}_parquet',f'{fdir}.json')
    # Writing metadata from all files in one directory to a single JSON file
    #should i write 1 large JSON file vs 1 for each file?
    with open(pjfile, 'w') as f:
        json.dump(metadata, f, ensure_ascii=False, indent=4)
    print(f'Wrote metadata to: {pjfile}')

os.chdir(cwd)
    

D:\MBSE\phd\research\work_folder\NASA
[WinError 183] Cannot create a file when that file already exists: 'D:\\MBSE\\phd\\research\\work_folder\\NASA\\Tail_687_1_parquet'
directory already exists...moving on.
Wrote metadata to: D:\MBSE\phd\research\work_folder\NASA\Tail_687_1_parquet\Tail_687_1.json


---

## Creating the full dataframe
However, I still need to figure out a solution for the 16hz time I'm creating that is linked to the inconsistent GMT time 

### Matlab format to dataframe format function

In [5]:
def mat_2_df(mdata, param, samplesize):
    '''This NASA matlab data is a dictionary of arrays of objects and this function
    takes in a single dictionary word and parses the data into a pandas dataframe
    '''
    #create an array filled with NaNs so we don't fill in data with bad data yet.
    d = np.empty(samplesize).reshape(-1,1)
    d[:] = np.NaN
    rate = mdata[param]['Rate'][0,0][0,0]
    i = int(16/rate)
    d[::i] = mdata[param]['data'][0,0].reshape(-1,1)

    return d
    

In [31]:
#need to loop through all of the directories and files again.
#wonder if this would be better to insert into the loops above?  it wouldn't be as modular

minimize_files = False

#Adding a custom built dirlist here so i can choose which ones I'm using
dirlist = ['Tail_660_7' ,'Tail_675_8', 'Tail_683_1', ]

for fdir in dirlist:
   
    #os.chdir(f'{cwd}/{fdir}')
    os.chdir(os.path.join(cwd,fdir))
    flist = glob.glob(f'*.mat')
    #filelist = ['687200104231404.mat']
    
    if minimize_files:
        #shuffle the files around in this list and only transform the first ones into parquet
        nkeep = 40
        np.random.shuffle(flist)    
        filelist = flist[:nkeep]
        delete_files  = flist[nkeep:]

        #delete the unlucky bastard files
        for f in delete_files:
            os.remove(f)
    else: 
        filelist = flist
    
    #write the parquet files into a new dir for parquet files
    pdir = os.path.join(cwd,f'{fdir}_parquet')
    
    #create the parquet directory unless it's already created
    try:
        os.mkdir(pdir)
    except OSError as error:
        print(error)
        print('directory already exists...moving on.')
    
    print(f'Converting flight data to dataframes from: {fdir}')
    
    for f in filelist:
        d = loadmat(f)


        hour   = d['GMT_HOUR']['data'][0,0].reshape(-1,1).astype(int)
       
        #skip the file if the plane never gets airborne for the duration of the file
        if min(hour) < 24:
            minute = d['GMT_MINUTE']['data'][0,0].reshape(-1,1).astype(int)
            second = d['GMT_SEC']['data'][0,0].reshape(-1,1).astype(int)
            
            #set time array as large as 16hz signal as NaNs then add actual time in GMT
            GMT = np.array(hour*3600 + minute*60 + second)
            
            #matches the fastest 16 Hz (adding an extra amount above 16Hz found in the data
            t_delta = 0.0625 + 0.5/549/16#seconds. 
            samplesize_16hz = d['FPAC']['data'][0,0].shape[0]
            
            #the first valid GMTsecs
            ti = np.argmin(GMT)
            t0 = GMT[ti]
            irange = ti+13
            
            #skip the file if it doesn't have much airborne data after the first GMT
            if irange < GMT.shape[0]: 
            
                #find the first time GMTsecs transitions to a new time, this is reliable GMT
                #it normally stays the same time for 6 seconds.
                i = min([i for i in range(ti,irange,1) if not t0 == GMT[i]])
                
                #GMTsecs is 2 Hz or 0.5 seconds
                timeoffset = 0.5*i
                
                #offset from the first reliable GMT
                starttime = GMT[i,0] - timeoffset 
                
                stoptime = starttime + t_delta*(samplesize_16hz-1) 
                
                #create a 16 Hz time column to line everything up and make it a pandas dataframe
                t16hz = np.linspace(starttime, stoptime, samplesize_16hz)
                df1 = pd.DataFrame(data={'time':t16hz})
                
                #adding GMT in seconds
                t = np.empty(samplesize_16hz).reshape(-1,1)
                t[:] = np.NaN
                t[::8] = GMT #every eighth element
                df1['GMTsecs'] = t
                
                #looping through all of the keys and creating a full Pandas DataFrame
                for k in d.keys():
                    #don't print the first "__xxxx__" parameters
                    if type(d[k]) == np.ndarray:
                       
                        df1[k] = mat_2_df(d, k, samplesize_16hz)
                #removing the first set of unreliable GMT data so it starts cleanly
                df1 = df1[i*8:-1]
                df1.reset_index(inplace=True)
                df1.pop('index')
                
                #removing filename extension
                file = f[0:-4]
                
                #writing to a parquet file in the parquet directory
                pfile = os.path.join(cwd,f'{fdir}_parquet',f'{file}.parquet')
                #overwriting file if it already exists
                df1.to_parquet(path=pfile, compression='gzip')
                print(f'Wrote parquet: {pfile}')
                
            else:
                print(f'Not enough airborne data: skipping file {f}')
            
        else:
            print(f'No airborne data: skipping file {f}')

        
os.chdir(cwd)

[Errno 17] File exists: '/ssd/MBSE/phd/research/work_folder/NASA/Tail_660_7_parquet'
directory already exists...moving on.
Converting flight data to dataframes from: Tail_660_7


/tmp/ipykernel_313448/2351370997.py:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[k] = mat_2_df(d, k, samplesize_16hz)
/tmp/ipykernel_313448/2351370997.py:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[k] = mat_2_df(d, k, samplesize_16hz)
/tmp/ipykernel_313448/2351370997.py:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

Wrote parquet: /ssd/MBSE/phd/research/work_folder/NASA/Tail_660_7_parquet/660200304161302.parquet


/tmp/ipykernel_313448/2351370997.py:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[k] = mat_2_df(d, k, samplesize_16hz)
/tmp/ipykernel_313448/2351370997.py:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[k] = mat_2_df(d, k, samplesize_16hz)
/tmp/ipykernel_313448/2351370997.py:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

Wrote parquet: /ssd/MBSE/phd/research/work_folder/NASA/Tail_660_7_parquet/660200303031318.parquet


...

Wrote parquet: /ssd/MBSE/phd/research/work_folder/NASA/Tail_660_7_parquet/660200306110609.parquet


/tmp/ipykernel_313448/2351370997.py:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[k] = mat_2_df(d, k, samplesize_16hz)
/tmp/ipykernel_313448/2351370997.py:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[k] = mat_2_df(d, k, samplesize_16hz)
/tmp/ipykernel_313448/2351370997.py:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

Wrote parquet: /ssd/MBSE/phd/research/work_folder/NASA/Tail_660_7_parquet/660200306111019.parquet


/tmp/ipykernel_313448/2351370997.py:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[k] = mat_2_df(d, k, samplesize_16hz)
/tmp/ipykernel_313448/2351370997.py:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[k] = mat_2_df(d, k, samplesize_16hz)
/tmp/ipykernel_313448/2351370997.py:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

Wrote parquet: /ssd/MBSE/phd/research/work_folder/NASA/Tail_660_7_parquet/660200306130551.parquet


## NaNs within the data due to different datarates are affecting the NN
this is an experiment to see if this works

In [19]:
[i for i in range(ti, irange, 1) if not t0 == GMT[i]]


[]

In [20]:
GMT.shape[0]

1928

In [28]:
f

'660200305210936.mat'

In [30]:
ti, irange

(0, 13)